In [1]:
# =============================================================================
# Cell 1: Reproducibility
# =============================================================================
SEED = 5
import os, random
import numpy as np
import tensorflow as tf
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-10-22 06:05:52.339122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761113152.538380      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761113152.597615      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Cell 2: Core Imports and Constants - ENHANCED VERSION
# =============================================================================
import json, math, os, gc
import cv2
from PIL import Image
import pickle
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (
    EfficientNetV2M, EfficientNetB5, InceptionV3, Xception, DenseNet169,
    DenseNet121, InceptionResNetV2, ResNet50
)
# Machine Learning
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.ensemble import VotingClassifier
import catboost as cb
import scipy
from scipy.special import softmax
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Enhanced visualization imports
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

# XAI Libraries
import shap
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm

# Jupyter
%matplotlib inline

# Constants - UPDATED BATCH SIZE FOR EFFICIENT MODELS
IMG_SIZE = 224
BATCH_SIZE = 16  # Reduced from 24 to 16 for EfficientNet models (they're memory intensive)
EPOCHS = 50
PATIENCE = 7

# Class labels for diabetic retinopathy
CLASS_LABELS = ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']

# Color palette for visualizations
COLORS = ['#2E8B57', '#FFD700', '#FF8C00', '#FF4500', '#DC143C']  # Green to Red gradient
CMAP_CUSTOM = LinearSegmentedColormap.from_list('custom', ['#ffffff', '#1f77b4', '#0d47a1'], N=256)

# GPU Memory Configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")

tf.config.optimizer.set_jit(True)
print("✅ XLA compilation enabled for memory efficiency")

# Kaggle input paths
APTOS_DIR = "/kaggle/input/aptos2019-blindness-detection"
PROCESSED_DATA_PATH = "/kaggle/input/pilise/preprocessed_aptos_data.npz"
TEST_CSV_PATH = "/kaggle/input/pilise/test_data.csv"
TRAIN_CSV_PATH = "/kaggle/input/pilise/train_data.csv"
print("✅ All imports completed successfully")

✅ GPU memory growth enabled for 1 GPU(s)
✅ XLA compilation enabled for memory efficiency
✅ All imports completed successfully


In [6]:
# Cell 3: Load Preprocessed Data
# =============================================================================
print("Loading preprocessed data...")
data = np.load(PROCESSED_DATA_PATH)
print("Available keys in preprocessed data:", list(data.keys()))

x_train_full = data['X_train']
y_train_full = data['y_train']
x_test = data['X_test']  
y_test = data['y_test']

train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

print(f"Training data shape: {x_train_full.shape}")
print(f"Training labels shape: {y_train_full.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")

# Store original labels for metrics calculation
y_test_original = y_test.copy() if len(y_test.shape) == 1 else np.argmax(y_test, axis=1)
y_train_original = y_train_full.copy() if len(y_train_full.shape) == 1 else np.argmax(y_train_full, axis=1)

print(f"Original label shapes - Train: {y_train_full.shape}, Test: {y_test.shape}")
if len(y_train_full.shape) == 1 or (len(y_train_full.shape) == 2 and y_train_full.shape[1] == 1):
    print("Converting training labels to one-hot encoding...")
    from tensorflow.keras.utils import to_categorical
    y_train_original = y_train_full.copy() if len(y_train_full.shape) == 1 else y_train_full.flatten()
    y_train_full = to_categorical(y_train_full, num_classes=5)
    print(f"New training labels shape: {y_train_full.shape}")

if len(y_test.shape) == 1 or (len(y_test.shape) == 2 and y_test.shape[1] == 1):
    print("Converting test labels to one-hot encoding...")
    from tensorflow.keras.utils import to_categorical
    y_test_original = y_test.copy() if len(y_test.shape) == 1 else y_test.flatten()
    y_test = to_categorical(y_test, num_classes=5)
    print(f"New test labels shape: {y_test.shape}")

print(f"Final shapes - Train: {y_train_full.shape}, Test: {y_test.shape}")
print(f"Label ranges - Train: [{y_train_full.min():.2f}, {y_train_full.max():.2f}], Test: [{y_test.min():.2f}, {y_test.max():.2f}]")

# Normalize pixel values
if x_train_full.max() > 1.0:
    print("Normalizing pixel values to [0,1]...")
    x_train_full = x_train_full.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
else:
    print("Data already normalized")
    x_train_full = x_train_full.astype('float32')
    x_test = x_test.astype('float32')

print("✅ Data loaded and preprocessed successfully")

Loading preprocessed data...
Available keys in preprocessed data: ['X_train', 'y_train', 'X_test', 'y_test']
Training data shape: (7220, 224, 224, 3)
Training labels shape: (7220, 5)
Test data shape: (733, 224, 224, 3)
Test labels shape: (733,)
Original label shapes - Train: (7220, 5), Test: (733,)
Converting test labels to one-hot encoding...
New test labels shape: (733, 5)
Final shapes - Train: (7220, 5), Test: (733, 5)
Label ranges - Train: [0.00, 1.00], Test: [0.00, 1.00]
Normalizing pixel values to [0,1]...
✅ Data loaded and preprocessed successfully


In [8]:
# =============================================================================
# Grad-CAM Visualization Script for Diabetic Retinopathy Models
# FULLY REWRITTEN: Avoids Sequential model issues by using Functional API
# =============================================================================

print("="*80)
print("GRAD-CAM VISUALIZATION SCRIPT - FUNCTIONAL API VERSION")
print("="*80)

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
import os

# =============================================================================
# Define Class Labels
# =============================================================================

CLASS_LABELS = {
    0: 'No DR',
    1: 'Mild',
    2: 'Moderate', 
    3: 'Severe',
    4: 'Proliferative DR'
}

print("✅ Class labels defined:")
for idx, label in CLASS_LABELS.items():
    print(f"   {idx}: {label}")

# =============================================================================
# Cell 1: Grad-CAM Implementation (FUNCTIONAL API APPROACH)
# =============================================================================

def get_last_conv_layer_name(model):
    """
    Automatically find the last convolutional layer in the model
    """
    # Check main model layers first
    for layer in reversed(model.layers):
        if 'Conv' in layer.__class__.__name__:
            return layer.name
    
    # Check inside base model if exists
    if hasattr(model.layers[0], 'layers'):
        for layer in reversed(model.layers[0].layers):
            if 'Conv' in layer.__class__.__name__:
                return layer.name
    
    raise ValueError("Could not find convolutional layer.")


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Generate Grad-CAM heatmap using direct layer access (NO Sequential issues)
    This approach extracts the base model and accesses layers directly
    """
    # Get the base model (assumes model structure: Sequential([base_model, other_layers]))
    if hasattr(model.layers[0], 'layers'):
        base_model = model.layers[0]
    else:
        base_model = model
    
    # Get the target conv layer
    try:
        conv_layer = base_model.get_layer(last_conv_layer_name)
    except:
        # Fallback if layer not in base model
        conv_layer = model.get_layer(last_conv_layer_name)
    
    # Create a new functional model: input -> [conv_output, final_predictions]
    # This completely bypasses the Sequential model structure
    try:
        # Try to create model using base model structure
        grad_model = Model(
            inputs=base_model.input,
            outputs=[conv_layer.output, base_model.output]
        )
        use_base = True
    except:
        # Fallback to full model
        grad_model = Model(
            inputs=model.input,
            outputs=[conv_layer.output, model.output]
        )
        use_base = False
    
    # Compute gradients
    with tf.GradientTape() as tape:
        # Forward pass through grad_model
        if use_base:
            # Need to complete the forward pass through remaining layers
            conv_outputs, base_predictions = grad_model(img_array)
            
            # Pass through remaining model layers (after base_model)
            x = base_predictions
            for layer in model.layers[1:]:
                x = layer(x)
            predictions = x
        else:
            conv_outputs, predictions = grad_model(img_array)
        
        # Get predicted class
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        
        # Get loss for target class
        class_channel = predictions[:, pred_index]
    
    # Compute gradients
    grads = tape.gradient(class_channel, conv_outputs)
    
    if grads is None:
        raise ValueError(f"Gradients are None for layer {last_conv_layer_name}")
    
    # Global average pooling
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Weight channels and sum
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    # Normalize
    heatmap = tf.maximum(heatmap, 0)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-10)
    
    return heatmap.numpy()


def overlay_gradcam(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    """
    Overlay Grad-CAM heatmap on original image
    """
    # Resize heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Convert to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # Convert image to uint8
    img_uint8 = np.uint8(255 * img)
    
    # Superimpose
    superimposed_img = heatmap * alpha + img_uint8
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')
    
    return superimposed_img


# =============================================================================
# Cell 2: Select 50 Images
# =============================================================================

def select_images_per_class(x_data, y_data, num_per_class=10, random_seed=42):
    """
    Select specified number of images per class
    """
    np.random.seed(random_seed)
    
    # Convert one-hot to integer if needed
    if len(y_data.shape) > 1:
        labels = np.argmax(y_data, axis=1)
    else:
        labels = y_data
    
    selected_indices = []
    
    for class_idx in range(5):
        class_indices = np.where(labels == class_idx)[0]
        
        if len(class_indices) >= num_per_class:
            selected = np.random.choice(class_indices, num_per_class, replace=False)
        else:
            selected = class_indices
            print(f"Warning: Only {len(class_indices)} images for class {class_idx}")
        
        selected_indices.extend(selected)
    
    selected_indices = np.array(selected_indices)
    selected_images = x_data[selected_indices]
    selected_labels = labels[selected_indices]
    
    return selected_indices, selected_images, selected_labels


print("\n📊 Selecting 50 images (10 per class)...")
selected_indices, selected_images, selected_labels = select_images_per_class(
    x_test, y_test, num_per_class=10, random_seed=42
)

print(f"✅ Selected {len(selected_indices)} images")
print(f"   Class distribution: {np.bincount(selected_labels)}")


# =============================================================================
# Cell 3: Model Configuration
# =============================================================================

MODEL_PATHS = {
    'EfficientNetV2M': '/kaggle/working/effnetv2m_final.h5',
    'DenseNet169': '/kaggle/working/densenet169_best.h5',
    'InceptionV3': '/kaggle/working/inceptionv3_final.h5',
    'EfficientNetB5': '/kaggle/working/effnetb5_final.h5'
}

# Check available models
available_models = {}
for model_name, model_path in MODEL_PATHS.items():
    if os.path.exists(model_path):
        available_models[model_name] = model_path
        print(f"✅ Found: {model_name}")
    else:
        print(f"⚠️  Not found: {model_name}")

if len(available_models) == 0:
    print("\n❌ No models found!")
else:
    print(f"\n✅ Found {len(available_models)} model(s)")


# =============================================================================
# Cell 4: Generate Grad-CAM
# =============================================================================

def generate_gradcam_for_model(model_name, model_path, images, labels):
    """
    Generate Grad-CAM visualizations
    """
    print(f"\n{'='*80}")
    print(f"Processing: {model_name}")
    print(f"{'='*80}")
    
    # Load model
    print(f"Loading model...")
    model = load_model(model_path)
    print(f"   ✓ Model loaded")
    
    # Print model structure for debugging
    print(f"   Model type: {type(model)}")
    print(f"   Number of layers: {len(model.layers)}")
    if hasattr(model.layers[0], 'layers'):
        print(f"   Base model: {type(model.layers[0])}")
        print(f"   Base model layers: {len(model.layers[0].layers)}")
    
    # Find conv layer
    try:
        last_conv_layer = get_last_conv_layer_name(model)
        print(f"✅ Using layer: {last_conv_layer}")
    except ValueError as e:
        print(f"❌ Error: {e}")
        return
    
    # Create directories
    output_dir = f'gradcam_{model_name.lower()}'
    original_dir = f'{output_dir}/original'
    overlay_dir = f'{output_dir}/overlay'
    combined_dir = f'{output_dir}/combined'
    
    os.makedirs(original_dir, exist_ok=True)
    os.makedirs(overlay_dir, exist_ok=True)
    os.makedirs(combined_dir, exist_ok=True)
    
    # Generate visualizations
    print(f"\n🔥 Generating Grad-CAM...")
    
    success_count = 0
    error_count = 0
    
    for idx, (img, true_label) in enumerate(zip(images, labels)):
        try:
            # Prepare image
            img_array = np.expand_dims(img, axis=0)
            
            # Get prediction
            preds = model.predict(img_array, verbose=0)
            pred_label = np.argmax(preds[0])
            confidence = preds[0][pred_label]
            
            # Generate Grad-CAM
            heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer, pred_index=pred_label)
            gradcam_img = overlay_gradcam(img, heatmap, alpha=0.4)
            
            # Filename
            base_filename = f'{idx+1:02d}_class{true_label}_pred{pred_label}'
            
            # Save original
            fig_orig = plt.figure(figsize=(5, 5))
            plt.imshow(img)
            plt.axis('off')
            plt.tight_layout(pad=0)
            plt.savefig(f'{original_dir}/{base_filename}_original.png', 
                       dpi=150, bbox_inches='tight', pad_inches=0)
            plt.close(fig_orig)
            
            # Save overlay
            fig_overlay = plt.figure(figsize=(5, 5))
            plt.imshow(gradcam_img)
            plt.axis('off')
            plt.tight_layout(pad=0)
            plt.savefig(f'{overlay_dir}/{base_filename}_overlay.png', 
                       dpi=150, bbox_inches='tight', pad_inches=0)
            plt.close(fig_overlay)
            
            # Combined visualization
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            axes[0].imshow(img)
            axes[0].set_title(f'Original\nTrue: {CLASS_LABELS[true_label]}', fontsize=10)
            axes[0].axis('off')
            
            axes[1].imshow(heatmap, cmap='jet')
            axes[1].set_title('Grad-CAM', fontsize=10)
            axes[1].axis('off')
            
            axes[2].imshow(gradcam_img)
            axes[2].set_title(f'Overlay\nPred: {CLASS_LABELS[pred_label]} ({confidence:.2%})', fontsize=10)
            axes[2].axis('off')
            
            correct = "✓" if true_label == pred_label else "✗"
            fig.suptitle(f'{model_name} - Image {idx+1}/50 {correct}', 
                        fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            plt.savefig(f'{combined_dir}/{base_filename}_combined.png', 
                       dpi=150, bbox_inches='tight')
            plt.close(fig)
            
            success_count += 1
            
            if (idx + 1) % 10 == 0:
                print(f"   ✓ Processed {idx+1}/50 ({success_count} successful)")
                
        except Exception as e:
            error_count += 1
            print(f"   ⚠️ Error on image {idx+1}: {str(e)}")
            continue
    
    print(f"\n✅ Completed: {success_count} successful, {error_count} errors")
    print(f"   Output: '{output_dir}/'")
    
    # Cleanup
    del model
    tf.keras.backend.clear_session()


# Process all models
for model_name, model_path in available_models.items():
    generate_gradcam_for_model(model_name, model_path, selected_images, selected_labels)


# =============================================================================
# Cell 5: Summary Comparison
# =============================================================================

print("\n" + "="*80)
print("CREATING SUMMARY COMPARISON")
print("="*80)

def create_summary_comparison(models_dict, images, labels, num_samples=5):
    """
    Create model comparison visualizations
    """
    print(f"\nCreating comparison for {num_samples} samples...")
    
    np.random.seed(42)
    
    # Select samples
    sample_indices = []
    for class_idx in range(min(5, num_samples)):
        class_mask = labels == class_idx
        class_imgs = np.where(class_mask)[0]
        if len(class_imgs) > 0:
            sample_indices.append(np.random.choice(class_imgs))
    
    # Load models
    loaded_models = {}
    conv_layers = {}
    
    print("Loading models...")
    for model_name, model_path in models_dict.items():
        loaded_models[model_name] = load_model(model_path)
        conv_layers[model_name] = get_last_conv_layer_name(loaded_models[model_name])
        print(f"   ✓ {model_name}")
    
    os.makedirs('gradcam_comparison', exist_ok=True)
    
    for sample_idx, img_idx in enumerate(sample_indices):
        print(f"   Comparison {sample_idx+1}/{len(sample_indices)}...")
        
        img = images[img_idx]
        true_label = labels[img_idx]
        
        num_models = len(loaded_models)
        fig, axes = plt.subplots(1, num_models + 1, figsize=(5 * (num_models + 1), 5))
        
        # Original
        axes[0].imshow(img)
        axes[0].set_title(f'Original\nTrue: {CLASS_LABELS[true_label]}', 
                         fontsize=11, fontweight='bold')
        axes[0].axis('off')
        
        # Grad-CAM from each model
        img_array = np.expand_dims(img, axis=0)
        
        for idx, (model_name, model) in enumerate(loaded_models.items()):
            preds = model.predict(img_array, verbose=0)
            pred_label = np.argmax(preds[0])
            confidence = preds[0][pred_label]
            
            heatmap = make_gradcam_heatmap(img_array, model, conv_layers[model_name], pred_index=pred_label)
            gradcam_img = overlay_gradcam(img, heatmap, alpha=0.4)
            
            axes[idx + 1].imshow(gradcam_img)
            axes[idx + 1].set_title(f'{model_name}\n{CLASS_LABELS[pred_label]} ({confidence:.1%})', 
                                   fontsize=11, fontweight='bold')
            axes[idx + 1].axis('off')
        
        fig.suptitle(f'Model Comparison - Sample {sample_idx + 1} (Class: {CLASS_LABELS[true_label]})', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        
        plt.savefig(f'gradcam_comparison/comparison_sample_{sample_idx + 1}.png', 
                   dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"✅ Saved to 'gradcam_comparison/'")
    
    # Cleanup
    for model in loaded_models.values():
        del model
    tf.keras.backend.clear_session()


# Create comparison
if len(available_models) > 1:
    create_summary_comparison(available_models, selected_images, selected_labels, num_samples=5)
elif len(available_models) == 1:
    print("⚠️  Only 1 model. Skipping comparison.")

print("\n" + "="*80)
print("✅ GRAD-CAM COMPLETED!")
print("="*80)
print(f"\n📁 Output directories:")
for model_name in available_models.keys():
    model_dir = f"gradcam_{model_name.lower()}"
    print(f"   - {model_dir}/")
if len(available_models) > 1:
    print(f"   - gradcam_comparison/")
print("\n🎉 Done!")


GRAD-CAM VISUALIZATION SCRIPT - FUNCTIONAL API VERSION
✅ Class labels defined:
   0: No DR
   1: Mild
   2: Moderate
   3: Severe
   4: Proliferative DR

📊 Selecting 50 images (10 per class)...
✅ Selected 50 images
   Class distribution: [10 10 10 10 10]
⚠️  Not found: EfficientNetV2M
⚠️  Not found: DenseNet169
✅ Found: InceptionV3
⚠️  Not found: EfficientNetB5

✅ Found 1 model(s)

Processing: InceptionV3
Loading model...
   ✓ Model loaded
   Model type: <class 'keras.src.models.sequential.Sequential'>
   Number of layers: 6
   Base model: <class 'keras.src.models.functional.Functional'>
   Base model layers: 311
✅ Using layer: conv2d_93

🔥 Generating Grad-CAM...
   ✓ Processed 10/50 (10 successful)
   ✓ Processed 20/50 (20 successful)
   ✓ Processed 30/50 (30 successful)
   ✓ Processed 40/50 (40 successful)
   ✓ Processed 50/50 (50 successful)

✅ Completed: 50 successful, 0 errors
   Output: 'gradcam_inceptionv3/'

CREATING SUMMARY COMPARISON
⚠️  Only 1 model. Skipping comparison.

✅ 